In [1]:
import os
import pandas as pd
from google.colab import drive

In [2]:
drive.mount("/content/drive/")
os.chdir("/content/drive/MyDrive/Holistic Insights/Hackathons/Turn Language into Action")

Mounted at /content/drive/


In [3]:
comments = pd.read_csv('data/comments_sentiment.csv')
print(comments.shape)
posts = pd.read_csv('data/posts.csv')
print(posts.shape)

(25441, 9)
(4957, 6)


In [4]:
posts['sentiment'] = posts['urn'].apply(lambda x: comments[comments.post_urn == x]['sentiment'].mean())
posts = posts[posts['sentiment'].notna()]
posts = posts[['text', 'sentiment']].reset_index(drop=True)

In [5]:
posts

,text,sentiment
0,With over 700 pregnancy-related deaths every y...,25.290000
1,"Parents, do you have questions about vaccinati...",-8.800000
2,Everyone should have the opportunity to be and...,7.796000
3,Why join our Sales team? Between your drive an...,76.800000
4,"This is what opening day is all about, coming ...",24.221111
...,...,...
3091,Take a look inside AutoNation Cadillac West Pa...,54.790000
3092,We are proud to announce that CuneXus has been...,13.556000
3093,That’s a wrap from Vegas! A huge thanks to our...,20.147273
3094,Our 10-year anniversary Boeing #ecoDemonstrato...,37.300000


In [8]:
posts[posts.sentiment > 0]

,text,sentiment
0,With over 700 pregnancy-related deaths every y...,25.290000
2,Everyone should have the opportunity to be and...,7.796000
3,Why join our Sales team? Between your drive an...,76.800000
4,"This is what opening day is all about, coming ...",24.221111
5,"Most of us know how important health is, but n...",15.211667
...,...,...
3091,Take a look inside AutoNation Cadillac West Pa...,54.790000
3092,We are proud to announce that CuneXus has been...,13.556000
3093,That’s a wrap from Vegas! A huge thanks to our...,20.147273
3094,Our 10-year anniversary Boeing #ecoDemonstrato...,37.300000


In [7]:
posts.sentiment.min(), posts.sentiment.max(), posts.sentiment.mean(), posts.sentiment.median()

(-88.0, 100.0, 19.267457205158497, 18.295)

In [ ]:
# !pip3 install autokeras

In [ ]:
import autokeras as ak
ak.__version__

'1.0.20'

In [ ]:
import numpy as np

split = int(posts.shape[0] * 0.8)
train = posts[:split]
test = posts[split:]

x_train = np.array(train.text)
y_train = np.array(train.sentiment)
x_test = np.array(test.text)
y_test = np.array(test.sentiment)

In [ ]:
print(x_train.shape)
print(y_train.shape)

(2476,)
(2476,)


In [ ]:
from autokeras import TextRegressor
text_reg = TextRegressor(overwrite=True, max_trials=1)

In [ ]:
text_reg.fit(
    x_train, 
    y_train,
    validation_split=0.15
)

Trial 1 Complete [00h 00m 30s]
val_loss: 246.99105834960938

Best val_loss So Far: 246.99105834960938
Total elapsed time: 00h 00m 30s
78/78 [==============================] - 2s 21ms/step - loss: 375.0161 - mean_squared_error: 375.0161


In [ ]:
y_pred = text_reg.predict(x_test)

20/20 [==============================] - 0s 5ms/step


In [ ]:
print(y_pred.flatten()[:20])

[19.36923  14.796199 14.438363 16.842333 21.7628   20.339525 20.597527
 17.535719 18.134338 19.592014 19.110443 18.63146  20.278915 17.108412
 18.46921  19.73393  19.603064 14.010542 17.042572 16.081285]


In [ ]:
print(y_test[:20])

[ 0.          0.89       -2.25714286 13.23       38.4         8.03666667
  9.84       33.79333333 46.59       21.595       3.9        41.29833333
 41.995      19.2        47.995       6.85       25.38428571 -2.7
 18.46       16.45      ]


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mean_squared_error(y_test, y_pred), mean_absolute_error(y_test, y_pred), r2_score(y_test, y_pred)

(268.50860408880555, 12.627640039164685, 0.011793008063395116)

In [ ]:
y_pred_binary = [1 if val >= 0 else 0 for val in y_pred.flatten()]
y_test_binary = [1 if val >= 0 else 0 for val in y_test]

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test_binary, y_pred_binary)

0.9483870967741935

In [ ]:
import tensorflow as tf
tf.__version__

'2.9.2'

In [ ]:
model = text_reg.export_model()
print(type(model))

<class 'keras.engine.functional.Functional'>


In [ ]:
try:
    model.save("model_autokeras", save_format="tf")
except Exception:
    model.save("model_autokeras.h5")

In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model("model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)
predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))

20/20 [==============================] - 0s 5ms/step
